In [ ]:
import pandas as pd
import matplotlib.ticker as ticker
import numpy as np
import sqlite3
from matplotlib import pyplot as plt

# Set pandas display options untuk menampilkan seluruh konten
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# df = pd.read_csv('../data/database/current/laptops_current_export.csv')
# df.head()

# Membaca file database SQLite untuk verifikasi
conn = sqlite3.connect('../data/database/current/laptops_current.db')
query = "SELECT * FROM products_current"
df = pd.read_sql_query(query, conn)

from pathlib import Path
# Robust DB locator: search upward for the database file to handle different working dirs
def locate_db(filename='laptops_current.db', max_up=6):
    cwd = Path.cwd().resolve()
    tried = []
    for i in range(max_up+1):
        p = cwd
        for _ in range(i):
            p = p.parent
        # Common candidate locations relative to the inspected directory
        candidates = [p / 'data' / 'database' / filename, p / 'web-scrape' / 'data' / 'database' / 'current' / filename, p / filename]
        for c in candidates:
            tried.append(str(c))
            if c.exists():
                return c
    raise FileNotFoundError("Database file not found. Tried:\n" + "\n".join(tried))

db_path = locate_db('laptops_current.db', max_up=6)
print(f'Resolved DB path: {db_path}')
conn = sqlite3.connect(str(db_path))
df = pd.read_sql_query("SELECT * FROM products_current", conn)
df.head(1)


In [ ]:
import pandas as pd
import matplotlib.ticker as ticker
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
from pathlib import Path

# =======================================================
# 1. PARAMETER PERUBAHAN
# =======================================================
TARGET_HASH = '00089e0fdc0a79fbcf4dc1ad9036ba7e92bd8a5e68f1411a5123b0768d3cf4f0'
NEW_PRICE_RAW = 15000000
NEW_PRICE_MILLIONS = NEW_PRICE_RAW / 1_000_000
DB_FILENAME = 'laptops_current.db'

print("="*50)
print(f"TARGET HASH: {TARGET_HASH}")
print(f"HARGA BARU (SIMULASI PERUBAHAN): Rp {NEW_PRICE_RAW:,}")
print("="*50)


# =======================================================
# 2. FUNGSI PENCARI PATH DB (Dari kode Anda, diperbaiki)
# =======================================================
def locate_db(filename=DB_FILENAME, max_up=6):
    """Mencari file database secara robust dengan menelusuri directory ke atas."""
    cwd = Path.cwd().resolve()
    tried = []
    # Loop untuk mencari ke atas (parent directories)
    for i in range(max_up + 1):
        p = cwd
        for _ in range(i):
            p = p.parent

        # Lokasi kandidat yang paling umum
        candidates = [
            p / 'data' / 'database' / 'current' / filename,
            p / 'data' / 'database' / filename,
            p / 'web-scrape' / 'data' / 'database' / 'current' / filename,
            p / filename
        ]
        
        for c in candidates:
            c = c.resolve() # Resolve path
            tried.append(str(c))
            if c.exists():
                return c

        if p == Path('/') or p == Path('.'): # Berhenti jika sudah di root
            break
        
    # Jika pencarian robust gagal, gunakan path absolut (asumsi environment Anda)
    fallback_path = Path(f"/home/wira/Documents/Project/web-scrape/data/database/current/{DB_FILENAME}")
    if fallback_path.exists():
        print(f"⚠️ Warning: Robust search failed. Using absolute fallback path: {fallback_path}")
        return fallback_path


    raise FileNotFoundError("Database file not found. Tried:\n" + "\n".join(tried))


# =======================================================
# 3. EKSEKUSI UPDATE SQL
# =======================================================
conn = None
try:
    # 3.1 Temukan path database
    db_path = locate_db(DB_FILENAME, max_up=6)
    print(f'Resolved DB path: {db_path}')

    # 3.2 Sambungkan ke database
    conn = sqlite3.connect(str(db_path))
    cursor = conn.cursor()

    # 3.3 Perintah SQL UPDATE
    # WHERE is_active = 1 penting untuk target SCD-2
    cursor.execute(f"""
        UPDATE products_current
        SET price_raw = ?,
            price_in_millions = ?
        WHERE product_hash = ? AND is_active = 1;
    """, (NEW_PRICE_RAW, NEW_PRICE_MILLIONS, TARGET_HASH))

    # 3.4 Commit perubahan
    conn.commit()
    
    rows_affected = cursor.rowcount
    
    if rows_affected > 0:
        print(f"\n✅ Berhasil mengubah {rows_affected} baris data.")
        
        # 3.5 Verifikasi Perubahan
        df_check = pd.read_sql_query(f"""
            SELECT product_hash, product_name, price_raw, price_in_millions, is_active 
            FROM products_current 
            WHERE product_hash = '{TARGET_HASH}'
        """, conn)
        print("\nVerifikasi Data Setelah Update:")
        print(df_check.to_string(index=False))

    else:
        print("⚠️ Produk tidak ditemukan atau tidak aktif (is_active=1). Pastikan hash sudah benar.")

except FileNotFoundError as e:
    print(f"❌ ERROR: File Database Tidak Ditemukan. {e}")
except sqlite3.OperationalError as e:
     # Ini adalah error 'unable to open database file' yang Anda alami
    print(f"\n❌ TERJADI ERROR FILE LOCK (OperationalError): {e}")
    print("SOLUSI: Mohon Coba 'Restart' Kernel VS Code/Jupyter, lalu jalankan kembali sel ini.")
except Exception as e:
    print(f"❌ Terjadi error tak terduga: {e}")

finally:
    if conn:
        conn.close()
        print("\nKoneksi ditutup.")

In [ ]:
# # Connect to laptops_history.db
def locate_db(filename='laptops_history.db', max_up=6):
    cwd = Path.cwd().resolve()
    tried = []
    for i in range(max_up+1):
        p = cwd
        for _ in range(i):
            p = p.parent
        # Common candidate locations relative to the inspected directory
        candidates = [p / 'data' / 'database' / filename, p / 'web-scrape' / 'data' / 'database' / 'history' / filename, p / filename]
        for c in candidates:
            tried.append(str(c))
            if c.exists():
                return c
    raise FileNotFoundError("Database file not found. Tried:\n" + "\n".join(tried))
db_path = locate_db('laptops_history.db', max_up=6)
print(f'Resolved DB path: {db_path}')
conn = sqlite3.connect(str(db_path))
df = pd.read_sql_query("SELECT * FROM products_history", conn)
df.head()

In [ ]:
df.count()

In [1]:
import sqlite3
from pathlib import Path
import pandas as pd

# =======================================================
# 1. PARAMETER PENCARIAN
# =======================================================
TARGET_HASH = '00089e0fdc0a79fbcf4dc1ad9036ba7e92bd8a5e68f1411a5123b0768d3cf4f0'
DB_FILENAME = 'laptops_history.db'

print("="*60)
print(f"Mencari data history untuk hash: {TARGET_HASH}")
print("="*60)


# =======================================================
# 2. FUNGSI PENCARI PATH DB (Revisi untuk history)
# =======================================================
def locate_db(filename=DB_FILENAME, max_up=6):
    """Mencari file database history secara robust."""
    cwd = Path.cwd().resolve()
    # Path history sering berbeda dari current, kita pastikan mencakup folder 'history'
    tried = []
    for i in range(max_up + 1):
        p = cwd
        for _ in range(i):
            p = p.parent

        candidates = [
            # Lokasi spesifik history (paling akurat)
            p / 'data' / 'database' / 'history' / filename,
            p / 'web-scrape' / 'data' / 'database' / 'history' / filename,
            # Lokasi fallback lain
            p / 'data' / 'database' / filename,
            p / filename
        ]
        
        for c in candidates:
            c = c.resolve()
            tried.append(str(c))
            if c.exists():
                return c
        
        if p == Path('/') or p == Path('.'):
            break
        
    fallback_path = Path(f"/home/wira/Documents/Project/web-scrape/data/database/history/{DB_FILENAME}")
    if fallback_path.exists():
        print(f"⚠️ Warning: Robust search failed. Using absolute fallback path: {fallback_path}")
        return fallback_path


    raise FileNotFoundError("Database history file not found. Tried:\n" + "\n".join(tried))


# =======================================================
# 3. EKSEKUSI QUERY SQL
# =======================================================
conn = None
try:
    # 3.1 Temukan path database history
    db_path = locate_db(DB_FILENAME, max_up=6)
    print(f'Resolved DB path: {db_path}')

    # 3.2 Sambungkan ke database
    conn = sqlite3.connect(str(db_path))
    
    # 3.3 Eksekusi query dengan filter hash
    query = f"""
        SELECT 
            history_id, product_hash, product_name, 
            price_raw, price_in_millions, is_active, 
            valid_from, valid_to
        FROM products_history
        WHERE product_hash = '{TARGET_HASH}'
        ORDER BY valid_from DESC
    """
    
    df_history = pd.read_sql_query(query, conn)

    if not df_history.empty:
        print(f"\n✅ Ditemukan {len(df_history)} riwayat untuk hash tersebut. (Hanya menampilkan kolom kunci):")
        # Menggunakan to_string untuk menghindari error 'tabulate'
        print(df_history.to_string(index=False))
        
        # Konfirmasi harga yang diuji
        old_price = df_history['price_raw'].iloc[0]
        print(f"\n⭐ Konfirmasi: Harga yang masuk ke HISTORY adalah Rp {old_price:,} (Simulasi Anda).")
    else:
        print("\n⚠️ Data riwayat (history) untuk hash ini tidak ditemukan.")


except FileNotFoundError as e:
    print(f"❌ ERROR: File Database Tidak Ditemukan. {e}")
except sqlite3.OperationalError as e:
    print(f"\n❌ TERJADI ERROR FILE LOCK (OperationalError): {e}")
    print("SOLUSI: Mohon Coba 'Restart' Kernel VS Code/Jupyter, lalu jalankan kembali sel ini.")
except Exception as e:
    print(f"❌ Terjadi error tak terduga: {e}")

finally:
    if conn:
        conn.close()
        print("\nKoneksi ditutup.")

Mencari data history untuk hash: 00089e0fdc0a79fbcf4dc1ad9036ba7e92bd8a5e68f1411a5123b0768d3cf4f0
Resolved DB path: /home/wira/Documents/Project/web-scrape/data/database/history/laptops_history.db

✅ Ditemukan 1 riwayat untuk hash tersebut. (Hanya menampilkan kolom kunci):
 history_id                                                     product_hash                                                                                                                                                                                                    product_name  price_raw  price_in_millions  is_active                valid_from                  valid_to
         59 00089e0fdc0a79fbcf4dc1ad9036ba7e92bd8a5e68f1411a5123b0768d3cf4f0 MSI Thin GF63 12UC Core i7 12650H Memori 8 GB SSD 4 TB RTX3050 Win 11 Pro (Intel Core i7 12650H, RAM 8 GB, SSD 4 TB, Vga Nvidia Geforce RTX3050 4 GB, LED 15.6" FHD 144 Hz, Windows 11 Profesional) Ready Stock   15000000               15.0          0 2025-11-29T14:30:44+00

In [2]:
import sqlite3
from pathlib import Path
import pandas as pd

# =======================================================
# 1. PARAMETER PENCARIAN
# =======================================================
TARGET_HASH = '00089e0fdc0a79fbcf4dc1ad9036ba7e92bd8a5e68f1411a5123b0768d3cf4f0'
# KITA AKAN CEK DATABASE CURRENT
DB_FILENAME = 'laptops_current.db' 

print("="*60)
print(f"Mencari data AKTIF di CURRENT DB untuk hash: {TARGET_HASH}")
print("="*60)


# =======================================================
# 2. FUNGSI PENCARI PATH DB
# =======================================================
def locate_db(filename=DB_FILENAME, max_up=6):
    """Mencari file database current secara robust."""
    cwd = Path.cwd().resolve()
    tried = []
    for i in range(max_up + 1):
        p = cwd
        for _ in range(i):
            p = p.parent

        candidates = [
            # Lokasi spesifik current
            p / 'data' / 'database' / 'current' / filename,
            p / 'web-scrape' / 'data' / 'database' / 'current' / filename,
            p / 'data' / 'database' / filename,
            p / filename
        ]
        
        for c in candidates:
            c = c.resolve()
            tried.append(str(c))
            if c.exists():
                return c
        
        if p == Path('/') or p == Path('.'):
            break
        
    fallback_path = Path(f"/home/wira/Documents/Project/web-scrape/data/database/current/{DB_FILENAME}")
    if fallback_path.exists():
        print(f"⚠️ Warning: Robust search failed. Using absolute fallback path: {fallback_path}")
        return fallback_path

    raise FileNotFoundError("Database current file not found. Tried:\n" + "\n".join(tried))


# =======================================================
# 3. EKSEKUSI QUERY SQL
# =======================================================
conn = None
try:
    # 3.1 Temukan path database current
    db_path = locate_db(DB_FILENAME, max_up=6)
    print(f'Resolved DB path: {db_path}')

    # 3.2 Sambungkan ke database
    conn = sqlite3.connect(str(db_path))
    
    # 3.3 Eksekusi query dengan filter hash dan status AKTIF
    query = f"""
        SELECT 
            product_hash, product_name, 
            price_raw, price_in_millions, is_active, 
            valid_from, valid_to
        FROM products_current
        WHERE product_hash = '{TARGET_HASH}' AND is_active = 1
    """
    
    df_current = pd.read_sql_query(query, conn)

    if not df_current.empty:
        print(f"\n✅ Ditemukan {len(df_current)} baris AKTIF untuk hash tersebut. (Hanya menampilkan kolom kunci):")
        print(df_current.to_string(index=False))
        
        current_price = df_current['price_raw'].iloc[0]
        
        print(f"\n⭐ Konfirmasi Harga Saat Ini: Rp {current_price:,}")
        
        if current_price == 14900000:
             print("🎉 SUCCESS: Harga SUDAH kembali normal (Rp 14,900,000) sesuai data scraper terbaru.")
        else:
             print("❓ PERHATIAN: Harga tidak sesuai yang diharapkan. Perlu pengecekan lebih lanjut.")

    else:
        print("\n⚠️ Data AKTIF untuk hash ini tidak ditemukan di products_current.db.")


except FileNotFoundError as e:
    print(f"❌ ERROR: File Database Tidak Ditemukan. {e}")
except sqlite3.OperationalError as e:
    print(f"\n❌ TERJADI ERROR FILE LOCK (OperationalError): {e}")
    print("SOLUSI: Mohon Coba 'Restart' Kernel VS Code/Jupyter, lalu jalankan kembali sel ini.")
except Exception as e:
    print(f"❌ Terjadi error tak terduga: {e}")

finally:
    if conn:
        conn.close()
        print("\nKoneksi ditutup.")

Mencari data AKTIF di CURRENT DB untuk hash: 00089e0fdc0a79fbcf4dc1ad9036ba7e92bd8a5e68f1411a5123b0768d3cf4f0
Resolved DB path: /home/wira/Documents/Project/web-scrape/data/database/current/laptops_current.db

✅ Ditemukan 1 baris AKTIF untuk hash tersebut. (Hanya menampilkan kolom kunci):
                                                    product_hash                                                                                                                                                                                                    product_name  price_raw  price_in_millions  is_active                valid_from valid_to
00089e0fdc0a79fbcf4dc1ad9036ba7e92bd8a5e68f1411a5123b0768d3cf4f0 MSI Thin GF63 12UC Core i7 12650H Memori 8 GB SSD 4 TB RTX3050 Win 11 Pro (Intel Core i7 12650H, RAM 8 GB, SSD 4 TB, Vga Nvidia Geforce RTX3050 4 GB, LED 15.6" FHD 144 Hz, Windows 11 Profesional) Ready Stock   14900000               14.9          1 2025-12-12T10:42:35+00:00     None

⭐ Konfirmas

In [4]:
import sqlite3
from pathlib import Path
import pandas as pd

# =======================================================
# 1. PARAMETER PENCARIAN
# =======================================================
# Kita asumsikan 108 baris terakhir di history adalah hasil dari Run ID 8 (50 + 58)
MAX_ROWS_TO_DISPLAY = 108 
DB_FILENAME = 'laptops_history.db'

print("="*70)
print(f"Menampilkan {MAX_ROWS_TO_DISPLAY} Riwayat Produk Terbaru (Price Updates & Discontinued)")
print("="*70)


# =======================================================
# 2. FUNGSI PENCARI PATH DB (Sama seperti sebelumnya)
# =======================================================
def locate_db(filename=DB_FILENAME, max_up=6):
    """Mencari file database history secara robust."""
    cwd = Path.cwd().resolve()
    tried = []
    for i in range(max_up + 1):
        p = cwd
        for _ in range(i):
            p = p.parent

        candidates = [
            p / 'data' / 'database' / 'history' / filename,
            p / 'web-scrape' / 'data' / 'database' / 'history' / filename,
            p / 'data' / 'database' / filename,
            p / filename
        ]
        
        for c in candidates:
            c = c.resolve()
            tried.append(str(c))
            if c.exists():
                return c
        
        if p == Path('/') or p == Path('.'):
            break
        
    fallback_path = Path(f"/home/wira/Documents/Project/web-scrape/data/database/history/{DB_FILENAME}")
    if fallback_path.exists():
        print(f"⚠️ Warning: Robust search failed. Using absolute fallback path: {fallback_path}")
        return fallback_path

    raise FileNotFoundError("Database history file not found. Tried:\n" + "\n".join(tried))


# =======================================================
# 3. EKSEKUSI QUERY SQL
# =======================================================
conn = None
try:
    # 3.1 Temukan path database history
    db_path = locate_db(DB_FILENAME, max_up=6)
    print(f'Resolved DB path: {db_path}')

    # 3.2 Sambungkan ke database
    conn = sqlite3.connect(str(db_path))
    
    # 3.3 Eksekusi query untuk mengambil N baris terakhir (diasumsikan N=108)
    query = f"""
        SELECT 
            history_id, product_hash, product_name, price_raw, 
            is_active, valid_from, valid_to
        FROM products_history
        ORDER BY history_id DESC
        LIMIT {MAX_ROWS_TO_DISPLAY}
    """
    
    df_history = pd.read_sql_query(query, conn)
    
    # Membalik urutan agar ID terendah (yang masuk duluan) ada di atas
    df_history = df_history.iloc[::-1]

    if not df_history.empty:
        print(f"\n✅ Berhasil mengambil {len(df_history)} baris.")
        print("Tabel di bawah adalah data yang ditutup (is_active=0) pada Run ID 8:")
        # Menggunakan to_string dan memotong hash/nama untuk keterbacaan
        df_display = df_history.copy()
        df_display['product_hash'] = df_display['product_hash'].str[:8] + '...'

        print(df_display.to_string(index=False, max_rows=MAX_ROWS_TO_DISPLAY + 1, max_colwidth=15))

    else:
        print("\n⚠️ Data riwayat (history) tidak ditemukan.")

except Exception as e:
    print(f"❌ Terjadi error: {e}")

finally:
    if conn:
        conn.close()
        print("\nKoneksi ditutup.")

Menampilkan 108 Riwayat Produk Terbaru (Price Updates & Discontinued)
Resolved DB path: /home/wira/Documents/Project/web-scrape/data/database/history/laptops_history.db

✅ Berhasil mengambil 108 baris.
Tabel di bawah adalah data yang ditutup (is_active=0) pada Run ID 8:
 history_id product_hash    product_name  price_raw  is_active      valid_from        valid_to
          1  effa42db... Lenovo V14 G...    9400000          0 2025-11-29T1... 2025-12-12T1...
          2  f4aa8011... DELL Inspiro...   14900000          0 2025-11-29T1... 2025-12-12T1...
          3  084c5e1c... Lenovo V14 G...    6300000          0 2025-11-29T1... 2025-12-12T1...
          4  3c29d10c... Lenovo V14 G...    7400000          0 2025-11-29T1... 2025-12-12T1...
          5  975a6520... DELL Inspiro...   18400000          0 2025-11-29T1... 2025-12-12T1...
          6  a93bf7e3... DELL Inspiro...   12900000          0 2025-11-29T1... 2025-12-12T1...
          7  5ef97ab0... DELL Inspiro...   10700000          0 2

# EDA

In [ ]:
df.describe()

### 1. Price Distribution

In [ ]:
# Tentukan range harga yang ingin dipertahankan
min_price = 0
max_price = 50  # sesuai dengan range maksimal Anda
bin_width = 5   # ukuran konsisten setiap bin (5 juta)

# Buat bins dengan ukuran konsisten menggunakan numpy.arange()
bins = np.arange(min_price, max_price + bin_width, bin_width)

# Buat labels yang sesuai dengan bins konsisten
bin_labels = []
for i in range(len(bins)-1):
    if i == 0:
        label = f'<{bins[i+1]:.0f}Jt'
    elif i == len(bins)-2:
        label = f'>{bins[i]:.0f}Jt'
    else:
        label = f'{bins[i]:.0f}-{bins[i+1]:.0f}Jt'
    bin_labels.append(label)

plt.figure(figsize=(12, 6))
n, bins, patches = plt.hist(df['price_in_millions'], bins=bins, alpha=0.7, 
                           color='skyblue', edgecolor='black', rwidth=0.7)

plt.title('Laptop Price Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Range Price (Rp)', fontsize=12)
plt.ylabel('Number of Products', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')

# Set custom x-tick labels (posisi di tengah setiap bin)
bin_centers = bins[:-1] + np.diff(bins)/2
plt.xticks(bin_centers, bin_labels)

# Tambahkan value labels di atas setiap bar
for i, (value, patch) in enumerate(zip(n, patches)):
    if value > 0:
        plt.text(patch.get_x() + patch.get_width()/2, value + 0.1, 
                f'{int(value)}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('../results/price_distribution.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
# Membuat boxplot harga laptop berdasarkan brand dan menampilkan detail outlier
asus_df = df[df['brand'] == 'Asus']

# asus_df['price_in_millions'].isnull().sum()
price_stats = asus_df['price_in_millions'].describe()
min_price = price_stats['min']
max_price = price_stats['max']
q1 = price_stats['25%']
q3 = price_stats['75%']
iqr = q3 - q1
median_price = price_stats['50%']

plt.figure(figsize=(10, 6))
ax = sns.boxplot(x='brand', y='price_in_millions', data=asus_df, palette='Set2')
plt.title('Laptop Price Asus', fontsize=14, fontweight='bold')
plt.xlabel('Brand', fontsize=12)
plt.ylabel('Price (in Millions)', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')

# Tampilkan nilai statistik pada plot
plt.text(0, min_price, f'Min: {min_price:,.2f}', color='blue', fontsize=10, ha='center', va='bottom')
plt.text(0, max_price, f'Max: {max_price:,.2f}', color='red', fontsize=10, ha='center', va='top')
plt.text(0, median_price, f'Median: {median_price:,.2f}', color='green', fontsize=10, ha='center', va='center')
plt.text(0, q1, f'Q1: {q1:,.2f}', color='purple', fontsize=10, ha='center', va='center')
plt.text(0, q3, f'Q3: {q3:,.2f}', color='orange', fontsize=10, ha='center', va='center')

plt.show()

In [ ]:
# Boxplot harga untuk semua brand dengan label statistik menggunakan Plotly
import plotly.graph_objects as go
import plotly.express as px

fig = px.box(df, x='brand', y='price_in_millions', color='brand', points='outliers',
              title='Laptop Price Distribution by Brand (Plotly)',
              labels={'price_in_millions': 'Price (in Millions)', 'brand': 'Brand'})

# Tambahkan label statistik pada setiap brand
brands = df['brand'].unique()
for i, brand in enumerate(brands):
    brand_df = df[df['brand'] == brand]['price_in_millions']
    if len(brand_df) == 0:
        continue
    min_price = brand_df.min()
    max_price = brand_df.max()
    q1 = brand_df.quantile(0.25)
    q3 = brand_df.quantile(0.75)
    median_price = brand_df.median()
    
    # Tambahkan annotation untuk setiap statistik
    # fig.add_annotation(x=brand, y=min_price, text=f'Min: {min_price:,.2f}', showarrow=False, font=dict(color='blue', size=10))
    # fig.add_annotation(x=brand, y=max_price, text=f'Max: {max_price:,.2f}', showarrow=False, font=dict(color='red', size=10))
    # fig.add_annotation(x=brand, y=median_price, text=f'Median: {median_price:,.2f}', showarrow=False, font=dict(color='green', size=10))
    # fig.add_annotation(x=brand, y=q1, text=f'Q1: {q1:,.2f}', showarrow=False, font=dict(color='purple', size=10))
    # fig.add_annotation(x=brand, y=q3, text=f'Q3: {q3:,.2f}', showarrow=False, font=dict(color='orange', size=10))

fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

#### Insight
- The Indonesian laptop market is dominated by the 10–20 million rupiah (mid-range) price segment.
- Products under 5 million are very few, indicating a limited but potential ultra-budget market.
- Premium laptops (>25 million) only account for a small portion of the market, suitable for branding rather than volume selling.
- Indonesian consumers are very price-sensitive, tending to look for the "sweet spot" between performance and price.
- The biggest opportunity for local manufacturers is in the low-mid range segment (5-10 million) with added value such as 512GB Storage, 16GB RAM, and good build quality.

### 2. Brand Distribution

In [ ]:
brand_counts = df['brand'].value_counts()
plt.figure(figsize=(10, 6))
sns.barplot(x=brand_counts.index, y=brand_counts.values)
plt.title("Brand Laptop Distribution")
plt.ylabel("Number of Produk")
plt.xlabel("Brand")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../results/brand_distribution.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()


#### Insight
- Lenovo and Asus dominate the Indonesian laptop market with the highest proportion of products (around 50% combined).
- Acer, HP, and MSI round out the top five, playing a strong role in the mid-range and gaming segments.
- Local brands (Axioo, ADVAN, Zyrex) are still small but have great potential in the student and low-budget segments.
- Apple and Dell maintain their premium image with a limited number of products but high prices.
- The market shows fierce competition in the mid-range and a gap in the ultra-budget segment — an opportunity for local brands to expand.

### 3. Processor Category Distribution

In [ ]:
df_processor_filtered = df[df['processor_category'] != 'Unknown Category']
plt.figure(figsize=(10,8))
sns.countplot(y='processor_category', data=df_processor_filtered, order=df_processor_filtered['processor_category'].value_counts().index)
plt.title('Processor Category Distribution')
plt.xlabel('Number of Product', fontsize=12)
plt.ylabel('Processor Category', fontsize=12)
plt.tight_layout()
plt.savefig('../results/processor_category_distribution.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

#### Insight
- Intel Core i5 and i7 processors dominate the majority of the laptop market in Indonesia.
- AMD Ryzen 5 & 7 are the main competitors in the mid-high segment with strong multi-core performance.
- Intel Core Ultra is emerging as the future trend in laptops (AI-ready, high efficiency).
- Low-end processors (Celeron, Pentium) have almost disappeared, signaling a shift in the market toward efficiency and high performance.
- Apple Silicon remains exclusive, cementing the MacBook's position in the premium segment.

### 4. Joint Distribution (Brand vs Processor Category)

In [ ]:
# --- Buat pivot table ---
pivot = pd.crosstab(df['brand'], df['processor_category']).fillna(0)

# --- Visualisasi heatmap sederhana dan jelas ---
plt.figure(figsize=(14, 10))

# --- Membuat heatmap dengan warna yang lebih kontras ---
heatmap = sns.heatmap(
    pivot,
    annot=True,         # tampilkan angka
    fmt='g',            # format angka sebagai integer
    cmap="YlOrRd",      # warna yang lebih kontras
    linewidths=0.5,     # garis pembatas antar sel
    cbar_kws={'label': 'Number of Products'},  # label legend warna
    annot_kws={'size': 8}  # ukuran font untuk angka
)

# --- Styling sederhana dan terbaca ---
plt.title("Combination Brand vs Processor Category", fontsize=14, weight="bold", pad=20)
plt.xlabel("Processor Category", fontsize=12)
plt.ylabel("Brand", fontsize=12)

# Rotasi label agar lebih mudah dibaca
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

# Sesuaikan layout agar tidak terpotong
plt.tight_layout()

# --- Simpan hasil visualisasi ---
plt.savefig("../results/brand_vs_processor_category_distribution.png", dpi=300, bbox_inches="tight", facecolor="white")
plt.show()

#### Insight
- Lenovo, Asus, and Acer dominate the entire processor spectrum — demonstrating a multi-segment strategy from low-end to premium
- Intel Core i5 and i7 are the most common categories in almost all brands → a strong indicator of Intel's dominance
- HP & MSI perform strongly with Ryzen 5/7 → AMD succeeds in the productivity and gaming segments
- Apple Silicon exclusive to the MacBook line, reinforcing its high-end positioning
- The new Core Ultra trend signals the future direction of AI-ready laptops, with Asus and Lenovo leading the way.


### 5. GPU Category Distribution

In [ ]:
total = len(df)
plt.figure(figsize=(10,8))
ax = sns.countplot(y='gpu_category', data=df, order=df['gpu_category'].value_counts().index, palette='viridis')
plt.title('GPU Category Distribution')
plt.xlabel('Number of Product', fontsize=12)
plt.ylabel('GPU Category', fontsize=12)

# Hide the legend since it's redundant with y-axis labels
plt.legend([],[], frameon=False)

for p in ax.patches:
    width = p.get_width()
    percentage = f'{100 * width / total:.1f}%'
    x = p.get_x() + width + (0.01 * total)
    y = p.get_y() + p.get_height() / 2

    ax.text(x, y, f'{int(width)} ({percentage})', va='center', ha='left', fontsize=11)

plt.xlim(0, plt.xlim()[1] * 1.2)
plt.tight_layout()
plt.savefig('../results/gpu_category_distribution.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

### Insight
- 43% of laptops in the Indonesian market use Intel Integrated Graphics, making it the top choice for the productivity segment.
- NVIDIA GeForce (High-End + Performance) dominates nearly 40% of market sales, proving the strong demand for gaming and creative laptops.
- AMD Integrated is available in the mid-range segment, offering efficient performance at a competitive price.
- Exclusive GPUs such as Apple Silicon and NVIDIA Quadro represent a small but stable premium segment.
- This pattern reflects the balanced ecosystem of the Indonesian laptop market between the needs for efficiency, performance, and price.

### 6. RAM Distribution

In [ ]:
# RAM Distribution Visualization

df_ram_filtered = df[df['ram'] != 'Unknown RAM'] # Filetered RAM with value is 'Unknown RAM'
 
plt.figure(figsize=(12,8))
sns.countplot(data=df_ram_filtered, x='ram', order=df_ram_filtered['ram'].value_counts().index)
plt.title('RAM Capacity Distribution')
plt.xlabel('RAM')
plt.ylabel('Number of Product')
plt.tight_layout
plt.xticks(rotation=45)
plt.savefig('../results/ram_distribution.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()


In [ ]:
df_ram_filtered['ram'].str.strip().value_counts()

### Insight
- 16GB RAM is now the most common and ideal configuration in the Indonesian laptop market.
- 8GB remains popular in the mid-range segment, replacing 4GB as the new standard.
- 32GB–64GB shows strong expansion in the gaming and creative professional markets
- Extreme RAM (≥96GB) is extremely rare and only appears in specialized workstation laptops.
- This shift reflects the increased demand for computing power and multitasking among modern laptop users.

### 7. Storage Distribution

In [ ]:
# Storage Distribution Visualization

df_storage_filtered = df[df['storage'] != 'Unknown Storage']

plt.figure(figsize=(12,8))
sns.countplot(data=df_storage_filtered, x='storage', order=df_storage_filtered['storage'].value_counts().index)
plt.title('Storage Capacity Distribution')
plt.xlabel('STORAGE')
plt.ylabel('Number of Product')
plt.tight_layout
plt.xticks(rotation=45)
plt.savefig('../results/storage_distribution.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

### Insight
- 512GB storage capacity has become the most dominant capacity and the new standard for modern laptops.
- The 1TB capacity ranks second, making it a favorite choice among gamers and professionals.
- Capacities of 2TB–4TB are becoming common in premium and creator-class laptops.
- Small storage (<256GB) is becoming increasingly rare and is only found in budget laptops.

### 8. Display Size Distribution

In [ ]:
# Display Distribution Visualization

df_display_filtered = df[df['display'] != 'Unknown']

plt.figure(figsize=(12,8))
sns.countplot(data=df_display_filtered, x='display', order=df_display_filtered['display'].value_counts().index)
plt.title('Display Size Distribution')
plt.xlabel('Display Size (inch)')
plt.ylabel('Number of Product')
plt.tight_layout
plt.xticks(rotation=45)
plt.savefig('../results/display_distribution.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

#### Insight
- 14 inches is the most dominant screen size and has become the main standard in the modern laptop market.
- 15.6″ still strong in the gaming and multimedia segment.
- 16″ shows a significant growth trend in premium and creative laptops
- Small sizes (<13″) are starting to be abandoned, while large sizes (≥17″) are only relevant in the workstation segment.
- The shift in trends shows that users are oriented towards visual comfort and productivity efficiency rather than just mobility.  

### 9. Price Corelation with RAM by GPU Category

In [ ]:
# Konversi nilai RAM ke angka (tanpa 'GB')
df_ram_filtered['ram_gb'] = df_ram_filtered['ram'].str.replace('GB', '', regex=False).astype(int)

# --- Setup Style ---
sns.set_theme(style="whitegrid", font_scale=1.1)
plt.figure(figsize=(10, 7))

# --- Scatterplot dengan Warna GPU Category ---
scatter = sns.scatterplot(
    data=df_ram_filtered,
    x="ram_gb",
    y="price_in_millions",
    hue="gpu_category",
    palette="Spectral",
    alpha=0.7,
    s=70,
    edgecolor="none"
)

# --- Tambahkan Trendline (Hubungan RAM vs Harga) ---
sns.regplot(
    data=df_ram_filtered,
    x="ram_gb",
    y="price_in_millions",
    scatter=False,
    color="black",
    line_kws={"linestyle": "--", "linewidth": 2, "alpha": 0.8}
)

# --- Penyesuaian Label & Tata Letak ---
plt.title("Price vs RAM Capacity by GPU Category", fontsize=16, weight="bold")
plt.xlabel("RAM Capacity (GB)", fontsize=12)
plt.ylabel("Price (in Million Rupiah)", fontsize=12)

# Tata letak legend di luar grafik
plt.legend(title="GPU Category", bbox_to_anchor=(1.05, 1), loc='upper left')

# --- Styling tambahan ---
plt.grid(True, linestyle='--', alpha=0.4)
plt.tight_layout()

# --- Simpan hasil visualisasi ---
plt.savefig("../results/visual_price_vs_ram_gpu.png", dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

### Insight
- Laptop prices increase significantly with increased RAM capacity, and are strongly influenced by the type of GPU used.
- Dedicated GPUs (NVIDIA/AMD) come with a price premium even though the RAM capacity is the same, while integrated GPUs are suitable for the low-mid range segment with competitive prices.

### 10. Median Price by Processor Category and GPU Category

In [ ]:
# Memfilter data Processor Category untuk menghapus 'Unknown Category'
df_unknown_filtered = df[df['processor_category'] != 'Unknown Category']
# Membuat pivot table median harga berdasarkan kategori prosesor dan kategori GPU
pivot_table = df_unknown_filtered.pivot_table(index='processor_category', columns='gpu_category', values='price_in_millions', aggfunc='median')

# Setup figure
plt.figure(figsize=(14, 8))
sns.set_theme(style="whitegrid", font_scale=1.0)

# Buat heatmap
sns.heatmap(
    pivot_table,
    annot=True, fmt=".1f", cmap="YlGnBu", linewidths=.5,
    cbar_kws={'label': 'Median Price (in Million Rupiah)'}
)

# Styling
plt.title("Median Laptop Price by Processor Category and GPU Category", fontsize=15, weight='bold')
plt.xlabel("GPU Category", fontsize=11)
plt.ylabel("Processor Category", fontsize=11)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()

# Simpan hasil
plt.savefig("../results/heatmap_cpu_gpu_price.png", dpi=300, bbox_inches='tight')
plt.show()

### Insight
- The most expensive laptop on average comes from a combination of Intel Core i9 with NVIDIA GeForce Mainstream GPU, priced at around 56.5M → These are typically high-end gaming laptops or creator laptops with extreme specifications — such as premium displays, large amounts of RAM, and advanced cooling.
- The most premium laptops for professional use are characterized by a combination of Intel Xeon and NVIDIA Quadro Workstation, with a median price of around 45.7M → This combination is commonly used for professional work such as 3D design, CAD, animation, and heavy computing, so it focuses more on stability and durability than style or design.
- Mid-range laptops typically use Intel Core i5, i7, or AMD Ryzen 5–7 processors with GPUs such as NVIDIA GeForce Performance or High-End.
→ Priced between 10 and 30 million rupiah, they are suitable for general users, content creators, and casual gamers who want high performance without paying a premium price.
- Entry-level laptops use processors such as Intel N-Series, Intel Pentium, or AMD Entry-Level with integrated GPUs such as Intel Integrated Graphics. → The median price is around 5–7 million rupiah, ideal for students or users who focus on light activities such as typing and browsing. 
- Apple Silicon laptops (M1, M2, M3) have a median price of around 22 million rupiah. → Although not as extreme as gaming laptops or workstations, Apple remains in the premium consumer category, thanks to the high performance and power efficiency characteristic of its own chips.
- Overall, the higher the class of processor and GPU used, the greater the increase in the median price of the laptop. However, a high price does not always mean "more professional," because some consumer-grade gaming laptops can actually be more expensive than workstations.

In [ ]:
# pd.set_option('display.max_columns', None)
df_unknown_display_filtered = df[df['display'] != 'Unknown']
pd.crosstab(df['gpu_category'], df_unknown_display_filtered['display'])
# pd.crosstab(df['gpu_category'], df['display']).plot(kind='bar', figsize=(12, 6))
# plt.title('GPU Category vs Display Size')
# plt.xlabel('GPU Category')
# plt.ylabel('Display Size')
# plt.xticks(rotation=45)
# plt.tight_layout()
# save_path = '/home/wira/Documents/Project/web-scrape/data/gpu_vs_display.png'
# plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
# plt.show()

In [ ]:
# import streamlit as st

# st.title('Laptop Market Analysis Dashboard')
# st.sidebar.header('Filters')

# # Interactive filters
# selected_brand = st.sidebar.multiselect('Select Brand', df['brand'].unique())
# price_range = st.sidebar.slider('Price Range',
#                                 min_value=0,
#                                 max_value=50000000,
#                                 value=(0, 20000000))

# # Dynamic Visualizations
# filtered_df = df[(df['brand'].isin(selected_brand)) &
#                  (df['price_raw'].between(price_range[0], price_range[1]))]

# # Show Interactive Charts
# st.plotly_chart(create_interactive_scatter(filtered_df))    

In [ ]:
df.head(1)

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def create_matplotlib_scatter(df):
    """Membuat scatter plot menggunakan matplotlib"""
    if df.empty:
        st.warning("No data available for selected filters")
        return
    
    plt.figure(figsize=(10, 6))
    
    # Buat scatter plot manual
    for brand in df['brand'].unique():
        brand_data = df[df['brand'] == brand]
        plt.scatter(brand_data['price_raw'], brand_data['ram'], 
                   label=brand, alpha=0.6, s=50)
    
    plt.xlabel('Harga (Rp)')
    plt.ylabel('RAM')
    plt.title('Price vs RAM by Brand')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Format x-axis untuk harga
    plt.gca().ticklabel_format(style='plain', axis='x')
    
    return plt

def create_simple_dashboard():
    """Dashboard sederhana tanpa plotly"""
    st.title('Laptop Market Analysis Dashboard')
    st.sidebar.header('Filters')
    
    # Load data Anda di sini
    # df = pd.read_csv('your_data.csv')
    
    # Convert ke juta dulu
    df['price_in_millions'] = df['price_raw'] / 1000000

    # Filters
    selected_brand = st.sidebar.multiselect(
        'Pilih Brand', 
        df['brand'].unique(),
        default=df['brand'].unique()[:3]
    )
    
    price_range = st.sidebar.slider(
        'Range Harga (Rp)', 
        min_value=0, 
        max_value=int(df['price_in_millions'].max()),
        value=(0, int(df['price_in_millions'].quantile(0.8)))
    )
    
    # Filter data
    filtered_df = df[
        (df['brand'].isin(selected_brand)) & 
        (df['price_in_millions'].between(price_range[0], price_range[1]))
    ]
    
    # Metrics
    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("Total Produk", len(filtered_df))
    with col2:
        st.metric("Harga Rata-rata", f"Rp {filtered_df['price_in_millions'].mean():,.0f}")
    with col3:
        st.metric("Brand Terpilih", len(selected_brand))
    
    # Visualizations dengan matplotlib
    st.subheader("Scatter Plot: Harga vs RAM")
    fig = create_matplotlib_scatter(filtered_df)
    if fig:
        st.pyplot(fig)
    
    # Histogram harga
    st.subheader("Distribusi Harga")
    fig2, ax2 = plt.subplots(figsize=(10, 4))
    ax2.hist(filtered_df['price_in_millions'], bins=20, alpha=0.7, color='skyblue')
    ax2.set_xlabel('Harga (Rp)')
    ax2.set_ylabel('Jumlah Produk')
    ax2.ticklabel_format(style='plain', axis='x')
    st.pyplot(fig2)
    
    # Data table
    st.subheader("Data Filtered")
    st.dataframe(filtered_df[['brand', 'processor_detail', 'ram', 'storage', 'price_in_millions']].head(10))

# Run dashboard
if __name__ == "__main__":
    create_simple_dashboard()